<a href="https://colab.research.google.com/github/silverstar0727/ML-Pipeline-Tutorial/blob/main/custom_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 설치 및 라이브러리 임포트

In [1]:
# 해당 셀을 실행한 후에 반드시 "런타임 다시시작"을 해주세요
!pip install -q kfp
!pip3 install --user kfp google-cloud-aiplatform matplotlib --upgrade -q
# 추가
!pip3 install --user google-cloud-aiplatform --upgrade -q
!pip3 install --user kfp google-cloud-pipeline-components --upgrade -q

     |████████████████████████████████| 235kB 27.2MB/s 
     |████████████████████████████████| 133kB 33.3MB/s 
     |████████████████████████████████| 645kB 39.8MB/s 
     |████████████████████████████████| 112kB 45.6MB/s 
     |████████████████████████████████| 1.8MB 42.1MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 1.0MB 50.8MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
ERROR: nbclient 0.5.3 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.
ERROR: google-cloud-bigquery 1.21.0 has requirement google-resumable-media!=0.4.0,<0.5.0dev,>

In [1]:
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component, ClassificationMetrics, Metrics)
from kfp.v2.google.client import AIPlatformClient

# 추가
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2.google import experimental

## GCP 연결

In [2]:
# gcp 연결
from google.colab import auth as google_auth

google_auth.authenticate_user()

# 경로변수 설정
아래 항목들을 본인 환경에 맞게 수정해주세요.

* PROJECT_ID = <프로젝트 ID>
* REGION = <리전>
* BUCKET_NAME = <bucket 이름>
* USER = <user 이름>

In [8]:
from datetime import datetime

PROJECT_ID = 'mlops-210515'
REGION = "us-central1"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "gs://pipeline-3313"

USER = "JeongMin-Do"
PIPELINE_ROOT = "{}/pipeline_root/{}".format(BUCKET_NAME, USER)
aiplatform.init(project=PROJECT_ID)

In [13]:
# create args list for trainer

hp_dict = '{"num_hidden_layers": 3, "hidden_size": 32, "learning_rate": 0.01, "epochs": 1, "steps_per_epoch": -1}'
data_dir = "gs://aju-dev-demos-codelabs/bikes_weather/"
TRAINER_ARGS = ["--data-dir", data_dir, "--hptune-dict", hp_dict]

# create working dir to pass to job spec
import time

ts = int(time.time())
WORKING_DIR = f"{PIPELINE_ROOT}/{ts}"

MODEL_DISPLAY_NAME = f"train_deploy{ts}"
print(TRAINER_ARGS)
print(WORKING_DIR)
print(MODEL_DISPLAY_NAME)

['--data-dir', 'gs://aju-dev-demos-codelabs/bikes_weather/', '--hptune-dict', '{"num_hidden_layers": 3, "hidden_size": 32, "learning_rate": 0.01, "epochs": 1, "steps_per_epoch": -1}']
gs://pipeline-3313/pipeline_root/JeongMin-Do/1625620264
train_deploy1625620264


In [14]:
@component
def training_op(input1: str):
    print("training task: {}".format(input1))

In [15]:
@kfp.dsl.pipeline(name="train-endpoint-deploy" + TIMESTAMP, pipeline_root=PIPELINE_ROOT)
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    serving_container_image_uri: str = "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-3:latest",
):

    train_task = training_op("model training")
    experimental.run_as_aiplatform_custom_job(
        train_task,
        worker_pool_specs=[
            {
                "containerSpec": {
                    "args": TRAINER_ARGS,
                    "env": [{"name": "AIP_MODEL_DIR", "value": WORKING_DIR}],
                    "imageUri": "gcr.io/google-samples/bw-cc-train:latest",
                },
                "replicaCount": "1",
                "machineSpec": {
                    "machineType": "n1-standard-16",
                    "accelerator_type": aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_V100,
                    "accelerator_count": 4,
                },
            }
        ],
    )

    model_upload_op = gcc_aip.ModelUploadOp(
        project=project,
        display_name=model_display_name,
        artifact_uri=WORKING_DIR,
        serving_container_image_uri=serving_container_image_uri,
    )
    model_upload_op.after(train_task)

    endpoint_create_op = gcc_aip.EndpointCreateOp(
        project=project,
        display_name="pipelines-created-endpoint",
    )

    model_deploy_op = gcc_aip.ModelDeployOp(  # noqa: F841
        project=project,
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        machine_type="n1-standard-4",
    )


In [16]:
compiler.Compiler().compile(
    pipeline_func = pipeline, package_path = "custom-pipeline.json"
)

api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

response = api_client.create_run_from_job_spec(
    job_spec_path="custom-pipeline.json", 
    parameter_values={"project": PROJECT_ID,},
)